# Purchase funnel

**Main task:** to analyse a click stream file; analyse data of payment confirmation page.

1. The first step is to open the file and count all the page types entries. We formatted it as a dictionary ‘result’.


In [24]:
import csv

file_path = r'C:\Books\result\data\click_stream1.csv'
result = dict()

with open(file_path, mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file, fieldnames=['ID','page','date'])
    for row in csv_reader:
        page = list(row.items())[1][1]
        result[page] = result.get(page, 0) + 1


2. As we can see, home page visited *90,400* potential clients. But only *452* not only visited payment page, but actually bought something. It will be *0.5%* of the basic customer amount. So the main problem should be localised: **we should find a page level where activity is decreasing.**

In [52]:
for key, value in result.items():
    print(f'{key}:', value)

print()
print('Overall payment confirmation rate:', (result['4_payment_confirmation_page'] / result['1_home_page']) * 100)


1_home_page: 90400
2_search_page: 45200
3_payment_page: 6030
4_payment_confirmation_page: 452

Overall payment confirmation rate: 0.5


3. First of all, we checked click stream according to the every month in the file. There were four month (from January till April). All the data is sorted and displayed below. As we can see, some global changes happend in February: click stream reduced in nearly 5,000 items for the search page. It also continued in March.

In [53]:
result_month = dict()

with open(file_path, mode='r') as csv_file:
    csv_reader_2 = csv.DictReader(csv_file, fieldnames=['ID','page','date'])
    for row in csv_reader_2:
        month = row['date'].split('-')[1]
        if month not in result_month:
            result_month[month] = dict()
        page = row['page']
        result_month[month][page] = result_month[month].get(page, 0) + 1
            

In [57]:
for key, value in sorted(result_month.items()):
    print(key, value, sep='\n')


01
{'1_home_page': 22600, '2_search_page': 13554, '3_payment_page': 2390, '4_payment_confirmation_page': 189}
02
{'1_home_page': 22600, '2_search_page': 13687, '3_payment_page': 2412, '4_payment_confirmation_page': 173}
03
{'1_home_page': 22600, '2_search_page': 8879, '3_payment_page': 631, '4_payment_confirmation_page': 44}
04
{'1_home_page': 22600, '2_search_page': 9080, '3_payment_page': 597, '4_payment_confirmation_page': 46}


4. The confirmation rate reduced as fourth times as it was in the first month.

In [60]:
for key in sorted(result_month.keys()):
    confirmation = round(result_month[key]['4_payment_confirmation_page'] / result_month[key]['1_home_page'] * 100, 2)
    print(f'Month: {key}; confirmation rate: {confirmation}%')


Month: 01; confirmation rate: 0.84%
Month: 02; confirmation rate: 0.77%
Month: 03; confirmation rate: 0.19%
Month: 04; confirmation rate: 0.2%


5. To clarify, that the problem was with the whole page system we also checked payment and confirmation pages cpmparing to the search page. Here the search page stands for the 100% of the visitors. The result repeats: main changes are clear in March, which influenced all the following data. So the problem is global.

In [58]:
for key in sorted(result_month.keys()):
    month_payment = round(result_month[key]['3_payment_page'] / result_month[key]['2_search_page'] * 100, 2)
    print(f'Month: {key}; payment rate: {month_payment}%')
    
    confirmation = round(result_month[key]['4_payment_confirmation_page'] / result_month[key]['2_search_page'] * 100, 2)
    print(f'Month: {key}; confirmation rate: {confirmation}%', end='\n\n')
    

Month: 01; payment rate: 17.63%
Month: 01; confirmation rate: 1.39%

Month: 02; payment rate: 17.62%
Month: 02; confirmation rate: 1.26%

Month: 03; payment rate: 7.11%
Month: 03; confirmation rate: 0.5%

Month: 04; payment rate: 6.57%
Month: 04; confirmation rate: 0.51%



6. To understand the problem we used cleack stream file with device data: some of the user visited pages with mobile devices and some from the desktop ones.

In [60]:
result_desktop = dict()
result_mobile = dict()
file_path_2 = r'C:\Books\result\data\click_stream2.csv'

with open(file_path_2, 'r') as csv_file_2:
    csv_reader_3 = csv.DictReader(csv_file_2, fieldnames=['ID','page','date','device'])
    for row in csv_reader_3:
        month = row['date'].split('-')[1]
        page = row['page']
        if row['device'] == 'Desktop':
            if month not in result_desktop:
                result_desktop[month] = dict()
            result_desktop[month][page] = result_desktop[month].get(page, 0) + 1
        else:
            if month not in result_mobile:
                result_mobile[month] = dict()
            result_mobile[month][page] = result_mobile[month].get(page, 0) + 1


7. We saw that the problem depends on optimisation of the website for each device type. For desktop devices it starts from the payment page, and for mobile ones – from search page. So the further work required frontend/backend code check, and the problem is not only with confirmation page.

In [64]:
print('Desktop device data', end='\n\n')
for key, value in sorted(result_desktop.items()):
    print(key, value, sep='\n')


Desktop device data

01
{'1_home_page': 15050, '2_search_page': 7529, '3_payment_page': 1221, '4_payment_confirmation_page': 60}
02
{'1_home_page': 15050, '2_search_page': 7632, '3_payment_page': 1191, '4_payment_confirmation_page': 54}
03
{'1_home_page': 15050, '2_search_page': 7370, '3_payment_page': 296, '4_payment_confirmation_page': 14}
04
{'1_home_page': 15050, '2_search_page': 7569, '3_payment_page': 302, '4_payment_confirmation_page': 22}


In [65]:
print('Mobile device data', end='\n\n')
for key, value in sorted(result_mobile.items()):
    print(key, value, sep='\n')
    

Mobile device data

01
{'1_home_page': 7550, '2_search_page': 6025, '3_payment_page': 1169, '4_payment_confirmation_page': 129}
02
{'1_home_page': 7550, '2_search_page': 6055, '3_payment_page': 1221, '4_payment_confirmation_page': 119}
03
{'1_home_page': 7550, '2_search_page': 1509, '3_payment_page': 335, '4_payment_confirmation_page': 30}
04
{'1_home_page': 7550, '2_search_page': 1511, '3_payment_page': 295, '4_payment_confirmation_page': 24}


# Conclusion

We achieved the following goals during the analysis:

1. Analysed click stream files.
2. Noticed the main activity problem.
3. Localised the activity problem with purchase funnel according to the time periods.
4. Localised the activity problem according to the device types.
5. Noticed, that the payment confirmation problem is not the main issue – payment page is broken for desktop devices and search page is broken for mobile devices.
5. Suggested the further check steps.
